## References
- https://www.kaggle.com/competitions/comp-mediavida-2-futurama
- https://www.kaggle.com/code/gonzalorecioc/example-code
- https://machinelearningmastery.com/multi-label-classification-with-deep-learning/
- https://keras.io/api/models/model_training_apis/
- https://keras.io/guides/transfer_learning/

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import imageio
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet152

## Load data

In [4]:
root_url = '../input/comp-mediavida-2-futurama/'
df = pd.read_csv(root_url + 'train_data.csv')
IMG_SHAPE = (135, 180)
FILE_SHAPE = (135, 180, 3)

In [5]:
def train_val_test_split(df, p_train, p_val, p_test):
    col_random_idx = "random_index"
    idx = list(range(len(df)))
    random.shuffle(idx)
    df[col_random_idx] = idx
    
    n_train = len(idx) * p_train
    n_val = len(idx) * p_val + n_train

    train_df = df[df[col_random_idx] < n_train]
    val_df = df[(df[col_random_idx] >= n_train) & (df[col_random_idx] < n_val)]
    test_df = df[df[col_random_idx] >= n_val]
    print(f"train_df: {train_df.shape}, val_df: {val_df.shape}, test_df: {test_df.shape}")
    return train_df, val_df, test_df

In [9]:
train_df, val_df, test_df = train_val_test_split(df, p_train=0.8, p_val=0.2, p_test=0)

In [11]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=5,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')
train_set = train_datagen.flow_from_dataframe(train_df,
                                              directory= root_url + "train_img/",
                                              x_col="file",
                                              y_col=["isLeela", "isFry", "isBender"],
                                              color_mode="rgb",
                                              target_size=IMG_SHAPE,
                                              batch_size=32,
                                              class_mode="raw",
                                              shuffle=False,
                                              )

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
val_set = val_datagen.flow_from_dataframe(val_df,
                                              directory= root_url + "train_img/",
                                              x_col="file",
                                              y_col=["isLeela", "isFry", "isBender"],
                                              color_mode="rgb",
                                              target_size=IMG_SHAPE,
                                              batch_size=32,
                                              class_mode="raw",
                                              shuffle=False,
                                              )

# Model

Using a ResNet trained on imagenet to do transfer learning.  
Using avg RMSE to replicate the LB metric despite being a classification problem.

In [12]:
l_rate = 1e-3
es = EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=10, restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_root_mean_squared_error',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00005)

base_model = ResNet152(weights='imagenet', include_top=False, input_shape=FILE_SHAPE, pooling='avg')
base_model.trainable = False

model=Sequential()
model.add(base_model)
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate = .2))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=l_rate), metrics=[RootMeanSquaredError()])

In [13]:
history = model.fit(train_set, epochs=100, validation_data=val_set, callbacks=[es, learning_rate_reduction])

Retraining the whole network with a lower learning rate.
Fitting the raw data without augmentation.

In [14]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
train_set = train_datagen.flow_from_dataframe(train_df,
                                              directory= root_url + "train_img/",
                                              x_col="file",
                                              y_col=["isLeela", "isFry", "isBender"],
                                              color_mode="rgb",
                                              target_size=IMG_SHAPE,
                                              batch_size=32,
                                              class_mode="raw",
                                              shuffle=False,
                                              )

es = EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=10, restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_root_mean_squared_error',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00005)

# Unfreeze base model
base_model.trainable = True
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=1e-4), metrics=[RootMeanSquaredError()])
history = model.fit(train_set, epochs=100, validation_data=val_set, callbacks=[es, learning_rate_reduction])

Final fit with remaining data

In [15]:
history = model.fit(val_set, epochs=20)

## Model understanding

In [13]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [14]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### FP/FN analysis

In [71]:
y_pred = model.predict(val_set)

In [72]:
y_diff = np.equal(val_set.labels, y_pred > 0.5)

In [73]:
def acc_per_class(y_diff):
    lee = y_diff[:,0]
    fry = y_diff[:,1]
    bender = y_diff[:,2]
    print(f"Accuracy Leela: {sum(lee) / len(lee)}")
    print(f"Accuracy Fry: {sum(fry) / len(fry)}")
    print(f"Accuracy Bender: {sum(bender) / len(bender)}")

In [74]:
acc_per_class(y_diff)

In [59]:
avg_rmse = np.mean(np.sqrt(np.mean(np.square(val_set.labels - y_pred), axis=0)))
avg_rmse

#### Test AVG MSE

In [ ]:
y_pred_test = model.predict(test_set)
avg_rmse = np.mean(np.sqrt(np.mean(np.square(test_set.labels - y_pred_test), axis=0)))
avg_rmse

#### See images

In [ ]:
failed_idx = np.where(y_diff.sum(axis=1) < 3)[0]

In [22]:
def show_img(img_name, df, train=True):  
    if train:
        img_path = "train_img/"
    else: 
        img_path = "test_img/"
    image = imageio.imread(root_url + img_path + img_name)
    
    labels = df[df["file"] == img_name]
    if train:
        lee = labels["isLeela"].values[0]
        fry = labels["isFry"].values[0]
        ben = labels["isBender"].values[0]
        plt.title(f'isLeela: {lee}, isFry: {fry}, isBender: {ben}')
    plt.imshow(image)

In [103]:
idx = 5
show_img(val_set.filenames[failed_idx[idx]], df)
print(f"y_pred: {y_pred[failed_idx[idx]] > 0.5}")

---

## Submission Pipeline

In [16]:
df_example = pd.read_csv(root_url + 'sample_submission.csv')
test_img_name_list = df_example["file"].to_list()
df_submission = pd.DataFrame({"file": test_img_name_list})

In [17]:
blind_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
blind_set = blind_datagen.flow_from_dataframe(df_submission,
                                              directory= root_url + "test_img/",
                                              x_col="file",
                                              color_mode="rgb",
                                              target_size=IMG_SHAPE,
                                              batch_size=64,
                                              class_mode=None,
                                              shuffle=False,
                                              )

In [18]:
y_pred_test = model.predict(blind_set)

df_submission["isLeela"] = y_pred_test[:, 0]
df_submission["isFry"] = y_pred_test[:, 1]
df_submission["isBender"] = y_pred_test[:, 2]

In [19]:
df_submission

In [24]:
show_img(blind_set.filenames[4], df, train=False)

In [20]:
df_submission.to_csv('submission_resnet_aug7.csv', index=False)